In [1]:
import keras
import tensorflow
import h5py
import numpy as np
import chowdhury_s_model_builder
import os

import tensorflow as tf
from keras.api.models import Sequential, Model, load_model
from keras.api.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, LSTM, Input, GlobalAveragePooling2D
from keras.api.optimizers import Adam
from keras import regularizers

2025-04-26 02:55:06.676856: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-26 02:55:06.926890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745607307.030428    9867 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745607307.057287    9867 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745607307.253561    9867 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Open the HDF5 file in read mode
with h5py.File("Data/Mid-Level_Perceptual_Features_Data.h5", "r") as hf:
    mid_level_train_data = hf["train"][:]  # Load the train dataset
    mid_level_label_data = hf["label"][:]  # Load the label dataset

print(mid_level_train_data.shape, mid_level_train_data.dtype)  # Check the shape and type
print(mid_level_label_data.shape, mid_level_label_data.dtype)

(5000, 256, 1292) float32
(5000, 7) int8


In [3]:
# Open the HDF5 file in read mode
with h5py.File("Data/Emotify_Data.h5", "r") as hf:
    emotify_train_data = hf["train"][:]  # Load the train dataset
    emotify_label_data = hf["label"][:]  # Load the label dataset

print(emotify_train_data.shape, emotify_train_data.dtype)  # Check the shape and type
print(emotify_label_data.shape, emotify_label_data.dtype)

(400, 256, 1292) float32
(400, 9) int8


In [4]:
mid_level_train_data = mid_level_train_data[..., np.newaxis]

print(mid_level_train_data.shape, mid_level_train_data.dtype)  # Check the shape and type
print(mid_level_label_data.shape, mid_level_label_data.dtype)

(5000, 256, 1292, 1) float32
(5000, 7) int8


In [5]:
emotify_train_data = emotify_train_data[..., np.newaxis]

print(emotify_train_data.shape, emotify_train_data.dtype)  # Check the shape and type
print(emotify_label_data.shape, emotify_label_data.dtype)

(400, 256, 1292, 1) float32
(400, 9) int8


In [6]:
mid_level_input_shape = mid_level_train_data.shape[1], mid_level_train_data.shape[2], mid_level_train_data.shape[3]
emotify_input_shape = emotify_train_data.shape[1], emotify_train_data.shape[2], emotify_train_data.shape[3]
print(mid_level_input_shape, emotify_input_shape)

(256, 1292, 1) (256, 1292, 1)


In [7]:
# with tensorflow.device('/GPU:0'):
#     input = Input(shape=(256, 1292, 1))
#     x =  Conv2D(64, (5, 5), strides=2, activation="relu", padding="valid")(input)
#     x =  BatchNormalization()(x)   
#     # 2nd Layer
#     x =  Conv2D(64, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 3rd Layer
#     x =  MaxPooling2D((2, 2))(x)
#     x =  Dropout(0.3)(x)    
#     # 4th Layer
#     x =  Conv2D(128, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 5th Layer
#     x =  Conv2D(128, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 6th Layer
#     x =  MaxPooling2D((2, 2))(x)
#     x =  Dropout(0.3)(x)    
#     # 7th Layer
#     x =  Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 8th Layer
#     x =  Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 9th Layer
#     x =  Conv2D(384, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 10th Layer
#     x =  Conv2D(512, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x)    
#     # 11th Layer
#     x =  Conv2D(256, (3, 3), strides=1, activation="relu", padding="same")(x)
#     x =  BatchNormalization()(x) 
#     # 12th Layer
#     # x = tfa.layers.AdaptiveAveragePooling2D(x)
#     x =  GlobalAveragePooling2D(keepdims=True)(x)
#     x =  Flatten()(x)
#     x = keras.layers.Dense(256)(x)
#     A2Mid2E_branch = keras.layers.Dense(7, activation="sigmoid")(x)
#     model = Model(inputs=input, outputs=A2Mid2E_branch, name="Mid-Level_Features")
#     model.summary()

In [8]:
model = load_model("model.keras")

I0000 00:00:1745607329.903951    9867 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1745607329.909581    9867 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [9]:
for layer in model.layers:
    if layer.trainable:
        print(layer.name, layer.__class__.__name__, layer.trainable)

input_layer InputLayer True
conv2d_4 Conv2D True
batch_normalization_4 BatchNormalization True
conv2d_5 Conv2D True
batch_normalization_5 BatchNormalization True
conv2d_6 Conv2D True
batch_normalization_6 BatchNormalization True
conv2d_7 Conv2D True
batch_normalization_7 BatchNormalization True
conv2d_8 Conv2D True
batch_normalization_8 BatchNormalization True
global_average_pooling2d GlobalAveragePooling2D True
flatten Flatten True
dense Dense True
dropout_inserted Dropout True
dense_1 Dense True


In [10]:
x = GlobalAveragePooling2D(name="InsertedGlobalAveragePooling2D")(model.layers[-5].output)
x = Dense(256, activation="relu", name="InsertedDense2")(x)
x = Dense(7, activation="sigmoid", name="InsertedDense1")(x)
model = Model(inputs=model.input, outputs=x)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 1292, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 126, 644, 64)   │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 644, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 126, 644, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 126, 644, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 322, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 322, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 63, 322, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 63, 322, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 63, 322, 128)   │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 63, 322, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 31, 161, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 31, 161, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 31, 161, 256)   │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 31, 161, 256)   │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 31, 161, 256)   │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 31, 161, 256)   │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 31, 161, 384)   │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 31, 161, 384)   │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 31, 161, 512)   │     1,769,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 31, 161, 512)   │         2,048 │
│ (BatchNormalization)            │                        │             

 Total params: 5,056,071 (19.29 MB)

 Trainable params: 4,791,175 (18.28 MB)

 Non-trainable params: 264,896 (1.01 MB)

In [11]:
for layer in model.layers[:-3]:
    layer.trainable = False
    
for layer in model.layers:
    if layer.trainable:
        print(layer.name, layer.__class__.__name__, layer.trainable)

InsertedGlobalAveragePooling2D GlobalAveragePooling2D True
InsertedDense2 Dense True
InsertedDense1 Dense True


In [12]:
# model.compile(optimizer=Adam(learning_rate=0.0005),
model.compile(optimizer=Adam(learning_rate=3.1250e-05),
                           loss='binary_crossentropy', metrics=["accuracy"]
                           )

In [13]:
model.fit(mid_level_train_data, mid_level_label_data,
                       batch_size=8,
                       epochs=10,
                       shuffle=True, verbose=1)

Epoch 1/10


I0000 00:00:1745607541.866878    9981 service.cc:152] XLA service 0x7f2598007ab0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745607541.868505    9981 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-04-26 02:59:02.021601: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1745607542.612069    9981 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-26 02:59:03.626685: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1258', 8 bytes spill stores, 8 bytes spill loads

2025-04-26 02:59:13.104588: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng11{k2=1,k3=0} for conv %cudnn-conv-bias-activation.27 = (f32[8,64,126,644]{

625/625 ━━━━━━━━━━━━━━━━━━━━ 241s 298ms/step - accuracy: 0.4761 - loss: 0.6241
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 189s 303ms/step - accuracy: 0.6392 - loss: 0.3888
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 182s 291ms/step - accuracy: 0.6433 - loss: 0.2318
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 185s 296ms/step - accuracy: 0.6599 - loss: 0.1552
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 181s 290ms/step - accuracy: 0.6812 - loss: 0.1128
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 209s 290ms/step - accuracy: 0.6813 - loss: 0.0818
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 183s 293ms/step - accuracy: 0.6885 - loss: 0.0655
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 181s 289ms/step - accuracy: 0.6907 - loss: 0.0507
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 181s 289ms/step - accuracy: 0.7084 - loss: 0.0429
Epoch 10/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 182s 292ms/step - accuracy: 0.7153 - loss: 0.0362


In [14]:
model.save("fine-tuned_model.keras")

In [15]:
# model.compile(optimizer=Adam(learning_rate=0.0005),
model.compile(optimizer=Adam(learning_rate=3.1250e-05),
                           loss='binary_crossentropy', metrics=["accuracy"]
                           )

In [16]:
model.fit(mid_level_train_data, mid_level_label_data,
                       batch_size=8,
                       epochs=20,
                       initial_epoch=10,
                       shuffle=True, verbose=1)

Epoch 11/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 192s 293ms/step - accuracy: 0.7102 - loss: 0.0288
Epoch 12/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 181s 290ms/step - accuracy: 0.7189 - loss: 0.0243
Epoch 13/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 180s 287ms/step - accuracy: 0.7226 - loss: 0.0194
Epoch 14/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 180s 287ms/step - accuracy: 0.7178 - loss: 0.0169
Epoch 15/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 182s 291ms/step - accuracy: 0.7277 - loss: 0.0154
Epoch 16/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 180s 287ms/step - accuracy: 0.7326 - loss: 0.0135
Epoch 17/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 178s 285ms/step - accuracy: 0.7089 - loss: 0.0134
Epoch 18/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 97s 155ms/step - accuracy: 0.7194 - loss: 0.0110
Epoch 19/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 173s 276ms/step - accuracy: 0.7093 - loss: 0.0104
Epoch 20/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 112s 178ms/step - accuracy: 0.7238 - loss: 0.0096


In [17]:
model.save("fine-tuned_model.keras")

In [18]:
# model.compile(optimizer=Adam(learning_rate=0.0005),
model.compile(optimizer=Adam(learning_rate=3.1250e-05),
                           loss='binary_crossentropy', metrics=["accuracy"]
                           )

In [19]:
model.fit(mid_level_train_data, mid_level_label_data,
                       batch_size=8,
                       epochs=30,
                       initial_epoch=20,
                       shuffle=True, verbose=1)

Epoch 21/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 120s 180ms/step - accuracy: 0.7318 - loss: 0.0101
Epoch 22/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 110s 176ms/step - accuracy: 0.7378 - loss: 0.0081
Epoch 23/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 147s 177ms/step - accuracy: 0.7269 - loss: 0.0079
Epoch 24/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 111s 177ms/step - accuracy: 0.7289 - loss: 0.0076
Epoch 25/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 112s 180ms/step - accuracy: 0.7332 - loss: 0.0069
Epoch 26/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 111s 178ms/step - accuracy: 0.7335 - loss: 0.0074
Epoch 27/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 111s 178ms/step - accuracy: 0.7278 - loss: 0.0067
Epoch 28/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 112s 180ms/step - accuracy: 0.7365 - loss: 0.0069
Epoch 29/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 111s 178ms/step - accuracy: 0.7375 - loss: 0.0067
Epoch 30/30
625/625 ━━━━━━━━━━━━━━━━━━━━ 111s 177ms/step - accuracy: 0.7436 - loss: 0.0063


In [20]:
model.save("fine-tuned_model.keras")

In [21]:
# model.compile(optimizer=Adam(learning_rate=0.0005),
model.compile(optimizer=Adam(learning_rate=3.1250e-05),
                           loss='binary_crossentropy', metrics=["accuracy"]
                           )

In [22]:
model.fit(mid_level_train_data, mid_level_label_data,
                       batch_size=8,
                       epochs=40,
                       initial_epoch=30,
                       shuffle=True, verbose=1)

Epoch 31/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 119s 177ms/step - accuracy: 0.7479 - loss: 0.0061
Epoch 32/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 112s 179ms/step - accuracy: 0.7423 - loss: 0.0057
Epoch 33/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 111s 177ms/step - accuracy: 0.7320 - loss: 0.0052
Epoch 34/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 98s 156ms/step - accuracy: 0.7275 - loss: 0.0058
Epoch 35/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 107s 172ms/step - accuracy: 0.7392 - loss: 0.0051
Epoch 36/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 99s 158ms/step - accuracy: 0.7425 - loss: 0.0063
Epoch 37/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 99s 159ms/step - accuracy: 0.7349 - loss: 0.0045
Epoch 38/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 99s 158ms/step - accuracy: 0.7342 - loss: 0.0042
Epoch 39/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 99s 159ms/step - accuracy: 0.7313 - loss: 0.0056
Epoch 40/40
625/625 ━━━━━━━━━━━━━━━━━━━━ 99s 159ms/step - accuracy: 0.7436 - loss: 0.0046
